<a href="https://colab.research.google.com/github/Ayaaba-Fatawu-Sumaila/Machine-Learning-Journey/blob/main/Home_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

rubyfath_my_files_path = kagglehub.dataset_download('rubyfath/my-files')

print('Data source import complete.')


KaggleApiHTTPError: 403 Client Error.

You don't have permission to access resource at URL: https://www.kaggle.com/datasets/rubyfath/my-files. The server reported the following issues: Permission 'datasets.get' was denied
Please make sure you are authenticated if you are trying to access a private resource or a resource requiring consent.

In [ ]:
#Importing libraries for dataframes,scientific computation and visualization
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
import numpy as np
import warnings

In [ ]:
warnings.filterwarnings('ignore') #Filtering outputs for cleaned results

In [ ]:
df = pd.read_csv("house_prices.csv") #reading the data into 'df' dataframes

## EXPLORATORY DATA ANALYSIS AND FEATURE ENGINEERING

In [ ]:
df.shape #Checking for the number of rows and columns in my dataframe

In [ ]:
df.sample(2) #Checking for 2 random sample rows of my dataframe

In [ ]:
df.info() #checking for my dataframes information

#### Cleaning and converting the Amount column to a numerical data

In [ ]:
df["Amount(in rupees)"].unique() #Checking for unique values in the  amount column

In [ ]:
#The amount has two different units, Lac and Cr

In [ ]:
df["Amount(in rupees)"] = df["Amount(in rupees)"].str.replace("Cr","*100",regex=False)
#Replacing and converting Cr to Lac
# 100 lac = 1Cr

In [ ]:
df["Amount(in rupees)"].unique()
#After replacing 100 lac = 1Cr

In [ ]:
df["Amount(in rupees)"] = df["Amount(in rupees)"].str.replace("Lac","",regex=False)
# Removing Lac from the values

In [ ]:
df["Amount(in rupees)"].unique()
#Current unique values in string

In [ ]:
#Assign the current column of "Amount(in rupees)" to amount
amount = df["Amount(in rupees)"]

In [ ]:
#Create empty list to store the evaluated string values to float by using a for loop
values = []
for i in amount:
    try:
        values.append(eval(i)) #eval(), function for mathematical calculation for converting string expressions to a folating values
    except:
        values.append(None) #For exception handling , by appending None if i doesn't exist in amount

In [ ]:
df["Amount(Lac)"] = values #Assigning the final convertion 'values' to the Amount(Lac) column

In [ ]:
df["Amount(Lac)"].unique() #Final amount values in float

In [ ]:
#Drop the df["Amount(in rupees)"] column
df.drop(["Amount(in rupees)"],axis=1,inplace=True)

In [ ]:
#Cleaning the price column by converting it from rupees to Lac

In [ ]:
df["Price (in rupees)"].unique()

In [ ]:
#Converting the price from rupees to Lac
df["Price (Lac)"] = df["Price (in rupees)"]/100_000  #1lac == 100,000rupees

In [ ]:
df["Price (Lac)"].unique()

In [ ]:
#Drop the "Price (in rupees)" column
df.drop(["Price (in rupees)"],axis=1,inplace=True)

#### Cleaning the Carpet area

In [ ]:
df["Carpet Area"].unique()

In [ ]:
df[df["Carpet Area"] == '164 sqyrd'] #Getting the index of the '164 sqyrd' for verification

##### Converting the square yard into square feet to have consistent units

In [ ]:
df["Carpet Area"] = df["Carpet Area"].str.replace("sqyrd","*9",regex=False)
#Converting square yard to square feet

In [ ]:
df["Carpet Area"] = df["Carpet Area"].str.replace("sqft","",regex=False).ffill()

In [ ]:
df["Carpet Area"].isnull().sum() #Checking for null values

In [ ]:
carpet = [] #Empty list for storing the converted values

In [ ]:
for u in df["Carpet Area"]:
    try:
        carpet.append(eval(u))
    except:
        carpet.append(None)

In [ ]:
df["Carpet_Area_sqft"] = carpet #assigning it to a new column

In [ ]:
#Drop the old column
df.drop(["Carpet Area"],axis=1,inplace=True)

In [ ]:
df["Carpet_Area_sqft"][[187488]] #Confirming for 164sqyrd to sqft. 1sqyrd == 9sqft

In [ ]:
#Done with the Carpet

#### Cleaning the Bathroom column

In [ ]:
df["Bathroom"].unique() #Checking for unique values in the Bathroom column

In [ ]:
df["Bathroom"].isnull().sum()*100/df.shape[0] #Checking for the percentage of Null values in Bathroom column

In [ ]:
df.groupby("Bathroom")["Price (Lac)"].value_counts()[:10]

In [ ]:
df["Bathroom"] = df["Bathroom"].fillna(method='ffill') #Using forward fill method for handling Null values

In [ ]:
df["Bathroom"].unique()

In [ ]:
df["Bathroom"] = df["Bathroom"].str.extract('(\d+)').astype(int) #Extracting the numerical values and converting to discrete values

In [ ]:
df["Bathroom"].unique()

In [ ]:
#Done with the Bathroom cleaning

#### Cleaning the Balcony column

In [ ]:
df["Balcony"].unique()

In [ ]:
df["Balcony"] = df["Balcony"].ffill() #Filling the null values with mode value

In [ ]:
df["Balcony"].unique()

In [ ]:
df["Balcony"] = df["Balcony"].str.extract('(\d+)').astype(int) #Extracting the numerical values and converting it to num

In [ ]:
df["Balcony"].unique()

In [ ]:
# Done cleaning df["Balcony"]

#### Cleaning the Super Area column

In [ ]:
df["Super Area"].unique()

In [ ]:
df["Super Area"] = df["Super Area"].bfill() #Backward fill for NaN data

In [ ]:
df["Super Area"] = df["Super Area"].str.replace("sqyrd","*9",regex=False) #Converting the data from square yard to square feet 1sqyrd = 9sqft

In [ ]:
df["Super Area"] = df["Super Area"].str.replace("sqft","",regex=False)

In [ ]:
super = []

In [ ]:
for area in df["Super Area"]:
    try:
        super.append(int(eval(area)))
    except:
        super.append(None)

In [ ]:
df["Super_Area_sqft"] = super

In [ ]:
df.drop("Super Area",axis=1,inplace=True)

In [ ]:
df["Super_Area_sqft"][131800] #For 9 * 140sqyrd == 1260

In [ ]:
df["Super_Area_sqft"].unique()

In [ ]:

#Done cleaning the "Super Area" column

In [ ]:
df.columns #Columns in the dataframe

In [ ]:
df.nunique() #Number of unique values in the columns of  the dataframe

In [ ]:
df.nunique()*100/df.shape[0] #Percentage of the unique values in the dataframe

In [ ]:
df.isnull().sum()*100/df.shape[0] #Percantage of the Null valueas in the dataframe

#### Dropping unecessary columns

In [ ]:
df.drop(["Plot Area","Dimensions","Car Parking","Society"],axis=1,inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop("Index",axis=1,inplace=True)

#### THE NUMERICAL DATA

In [ ]:
numerical_col = df.select_dtypes(exclude='object')[:5]

#### CATEGORICAL DATA

In [ ]:
categorical_col = df.select_dtypes(include='object')

### OUTLIERS REMOVAL USING ZSCORE,IQR AND BOXPLOT

#### Describe data

In [ ]:
categorical_col.describe() #Statistical summmary of both the categorical and text columns

In [ ]:
numerical_col.describe() # Statistical summmary of the numerical data columns

#### CHECKING THE FREQUENCIES OF AMOUNT OF DIFFERENT FURNISHING FEATURES

In [ ]:
df.Furnishing.unique()

In [ ]:
furnish_amount = pd.melt(df, id_vars="Furnishing",value_vars="Amount(Lac)") #For getting the "Furnishing" and "Amount(Lac)" columns

In [ ]:
furnish_amount[:4] #First four rows

In [ ]:
# "Bar Graph of Amount(Lac) against Furnishing"
plt.grid()
sns.barplot(x=furnish_amount["Furnishing"],y=furnish_amount["value"])
plt.title("Bar Graph of Amount(Lac) against Furnishing",fontsize=15)
plt.xlabel("Furnishing",fontsize=10)
plt.ylabel("Amount(Lac)",fontsize=10)
plt.show()

In [ ]:
#BOXPLOT OF NUMERICAL COLUMNS

In [ ]:
plt.figure(figsize=(15,10))
plt.grid()
sns.boxplot(numerical_col)
plt.title("BOXPLOT OF NUMERICAL COLUMNS",fontsize=20)
plt.show()

#### THE NUMBER OF BALCONY SHOULD NOT BE MORE THAN TWICE THE NUMBER OF BATHROOMS

In [ ]:
df.query("Balcony > Bathroom*2").shape

In [ ]:
#Drop the columns with Balcony > 2*Bathroom
mask_balcony = df["Balcony"] < df["Bathroom"]*2
df = df[mask_balcony]

In [ ]:
df.shape[0]-df.query("Balcony > Bathroom*2").shape[0] #Remaining rows

### Using zcsore for outliers removal

In [ ]:
df["Z_score"] = zscore(df["Bathroom"]) #zscore for number of bathrooms

In [ ]:
df["Z_score"].head() #zscore values

In [ ]:
df["Z_score"].min(),df["Z_score"].max()

##### Using a deviation of 3 and -3 intervals of std

In [ ]:
df = df[(df.Z_score > -3) & (df.Z_score <= 3)]

In [ ]:
df.shape

In [ ]:
#Dropping the z_score column
df.drop("Z_score",axis=1,inplace=True)

In [ ]:
df.sample() #Random sample of the data

### Using IQR for outlier removal, for Price and Amount

In [ ]:
Q1p,Q3p = df["Price (Lac)"].quantile([0.25,0.75]) #25% and 75% percentiles of the Price column

In [ ]:
IQR = (Q3p - Q1p)

In [ ]:
price_lower_limit = Q1p - IQR*1.5 #Lower limit of the price
price_upper_limit = Q3p + IQR*1.5 #Upper limit of the price

In [ ]:
round(price_lower_limit,3),round(price_upper_limit,3)

In [ ]:
mask_price = ((df["Price (Lac)"] > price_lower_limit) & (df["Price (Lac)"] < price_upper_limit))

In [ ]:
df = df[mask_price]

#### Using a boxplot to clean the Carpet_Area_sqft and	Super_Area_sqft columns

##### Cleaning the Carpet_Area_sqft column

In [ ]:
df["Carpet_Area_sqft"].min(),df["Carpet_Area_sqft"].mean(),df["Carpet_Area_sqft"].max()

In [ ]:
#Before outiers removal
plt.grid(visible=True)
plt.title("Boxplot of Carpet Area in Sqft",fontsize=15)
sns.boxplot(df["Carpet_Area_sqft"])
plt.yscale("log")

In [ ]:
mask_carpet = (df["Carpet_Area_sqft"] < 2200) & (df["Carpet_Area_sqft"] > 100)

In [ ]:
df = df[mask_carpet]

In [ ]:
#After outiers removal
plt.grid(visible=True)
plt.title("Boxplot of Carpet Area in Sqft",fontsize=15)
sns.boxplot(df["Carpet_Area_sqft"])
plt.yscale("log")

##### Cleaning the Super_Area_sqft column

In [ ]:
df["Super_Area_sqft"].min(),df["Super_Area_sqft"].mean(),df["Super_Area_sqft"].max()

In [ ]:
#Before outliers removal
plt.title("Boxplot of Carpet Area in Sqft",fontsize=15)
df.Super_Area_sqft.plot(kind='box')
plt.show()

In [ ]:
mask_super_area = (df.Super_Area_sqft > 100) & (df.Super_Area_sqft < 2200)

In [ ]:
df = df[mask_super_area]

In [ ]:
#after outliers removal
plt.title("Boxplot of Carpet Area in Sqft",fontsize=15)
df.Super_Area_sqft.plot(kind='box')
plt.show()

## VISUALISATION ANALYSIS OF THE DATA

In [ ]:
df.columns

#### USING HISTOGRAM TO CHECK FOR FREQUENCIES

In [ ]:
#Histogram of Super_Area_sqft
plt.hist(df.Super_Area_sqft)
plt.title("Super Area Square Feet",fontsize=15)
plt.xlabel("Super Area(sqft)",fontsize=10)
plt.ylabel("Frequency",fontsize=10)
plt.yscale("log")
plt.show()

In [ ]:
#Histogram of Super_Area_sqft
plt.hist(df.Carpet_Area_sqft)
plt.title("Carpet Area Square Feet",fontsize=15)
plt.xlabel("Carpet Area(sqft)",fontsize=10)
plt.ylabel("Frequency",fontsize=10)
plt.yscale("log")
plt.show()

In [ ]:
#Histogram of Price (Lac)
plt.hist(df["Price (Lac)"])
plt.title("Price In Lac",fontsize=15)
plt.xlabel("Price (Lac)",fontsize=10)
plt.ylabel("Frequency",fontsize=10)
plt.yscale("log")
plt.show()

In [ ]:
#Histogram of Amount(Lac)
plt.hist(df["Amount(Lac)"])
plt.title("Amount In Lac",fontsize=15)
plt.xlabel("Amount(Lac)",fontsize=10)
plt.ylabel("Frequency",fontsize=10)
plt.yscale("log")
plt.show()

In [ ]:
#From the above plot it shows that the value at 2500 is an outlier

In [ ]:
#Drop
mask_2500 = df["Amount(Lac)"] < 1400

In [ ]:
df = df[mask_2500]

In [ ]:
plt.hist(df["Amount(Lac)"])
plt.title("Amount In Lac",fontsize=15)
plt.xlabel("Amount(Lac)",fontsize=10)
plt.ylabel("Frequency",fontsize=10)
plt.yscale("log")
plt.show()

#### Using scatter plot to check for relationship between columns

In [ ]:
#Correlation between Price (Lac) and Amount(Lac)
corr = round(df["Price (Lac)"].corr(df["Amount(Lac)"]),3)
sns.regplot(x=df["Price (Lac)"],y=df["Amount(Lac)"],label=corr)
plt.title("Correlation between Price and Amount",fontsize=15)
plt.xlabel("Price (Lac)",fontsize=10)
plt.ylabel("Amount(Lac)",fontsize=10)
plt.legend()
plt.show()

In [ ]:
#Correlation between Balcony and Bathroom
corr2 = round(df["Balcony"].corr(df["Bathroom"]),3)
sns.regplot(x=df["Balcony"],y=df["Bathroom"],label=corr2,marker='+',color='g')
plt.title("Correlation between Balcony and Bathroom",fontsize=15)
plt.xlabel("Balcony",fontsize=10)
plt.ylabel("Bathroom",fontsize=10)
plt.legend()
plt.show()

In [ ]:
#Correlation between Carpet_Area_sqft and Bathroom
corr2 = round(df["Carpet_Area_sqft"].corr(df["Super_Area_sqft"]),3)
sns.regplot(x=df["Carpet_Area_sqft"],y=df["Super_Area_sqft"],label=corr2,marker='d',color='r')
plt.title("Correlation between Carpet Area and Super Area",fontsize=15)
plt.xlabel("Carpet Area(sqft)",fontsize=10)
plt.ylabel("Super Area(sqft)",fontsize=10)
plt.legend()
plt.show()

#### USING HEATMAP TO CHECK THE CORRELATION OF THE NUMERICAL COLUMNS OF THE DATA

In [ ]:
numerical_cols = df.select_dtypes(exclude='object').corr()

In [ ]:
#Using correlation heatmap to check the relationship of numerical columns

In [ ]:
plt.figure(figsize=(15,10))

sns.heatmap(numerical_cols,annot=True)
plt.title("CORRELATION BETWEEN NUMERICAL COLUMNS",fontsize=20)
plt.show()

### Plot relational pair plot to check relationship

In [ ]:
num_col = df.select_dtypes(exclude="object")

In [ ]:

sns.pairplot(num_col)
plt.show()

In [ ]:
num_col.columns #Choosing numerical columns with strong relationship


In [ ]:
specific_num_cols = df[["Amount(Lac)","Price (Lac)","Carpet_Area_sqft","Super_Area_sqft"]]

In [ ]:
#HIGHLY CORRELATED NUMERICAL COLUMNS
sns.pairplot(specific_num_cols)
plt.show()

In [ ]:
#Joint plot for "Carpet_Area_sqft","Super_Area_sqft"
sns.jointplot(x=df["Carpet_Area_sqft"],y=df["Super_Area_sqft"])
#plt.title("Joint Plot Between Carpet_Area_sqft and Super_Area_sqft")
plt.show()

In [ ]:
#Joint plot for Price (Lac) and Amount(Lac)
sns.jointplot(x=df["Price (Lac)"],y=df["Amount(Lac)"])
#plt.title("Joint Plot Between Price (Lac) and Amount(Lac)",fontsize=15)
plt.show()

### USING PREPROCESSING TECHNIQUES TO HANDLE NULL , TEXT, CATEGORICAL AND NUMERICAL VALUES

#### Saving my cleaned data

In [ ]:
df.to_csv("cleaned_house_price.csv",index=False)

In [ ]:
dff = pd.read_csv("cleaned_house_price.csv") #Starting to bbuild my model with my new data

In [ ]:
dff.shape

In [ ]:
dff.shape

In [ ]:
dff.isnull().sum()*100/dff.shape[0]

In [ ]:
#dff =dff.ffill() #Filling Null values using forward fill method

In [ ]:
#Due to storage drop all Null rows
dff.dropna(inplace=True)

In [ ]:
dff.sample()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### PREPROCESSING, PIPELINE AND MODEL BUILDING

In [ ]:
from sklearn.preprocessing import StandardScaler #For numerical features
from sklearn.preprocessing import OneHotEncoder #For categorical features
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer #For text features
from sklearn.impute import SimpleImputer #For Null values handling

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [ ]:
#Regression models

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

In [ ]:
#Model selection and Evaluation models
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.compose import ColumnTransformer

#### DIVIDE THE DATA INTO TRAIN(70%) ,VALIDATE(15%) AND TESTING(15%)

In [ ]:
dff.columns

In [ ]:
#Dividing the data into train,validate and test

In [ ]:
X = dff.drop(["Amount(Lac)"],axis=1)
y = dff["Amount(Lac)"]

In [ ]:
X[:3]

In [ ]:
X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,random_state=30)

In [ ]:
#Divide the X_temp and y_temp into validation and testing data into equal size 50% each

In [ ]:
X_val,X_test,y_val,y_test = train_test_split(X_temp,y_temp,test_size=0.5,random_state=30)

In [ ]:
#divide data into text,categorical and numerical arrays[ "Amount" taarget column]

In [ ]:
num_columns = ["Bathroom","Balcony","Carpet_Area_sqft","Price (Lac)","Super_Area_sqft"]

cat_columns = ["location","Transaction","Furnishing","facing","Ownership"]


#### Creating the pipeline

In [ ]:
#Numericals
num_pipe = Pipeline(steps=[
    ("impute",SimpleImputer(strategy='mean')),
    ("scale",StandardScaler())
])

In [ ]:
# Categorical
cat_pipe = Pipeline(steps=[
    ("impute",SimpleImputer(strategy='most_frequent')),
    ("one-hot-encoder",OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Text columns preprocessing
text_pipe1 = Pipeline(steps=[

    ("text_count",CountVectorizer())

])
text_pipe2 = Pipeline(steps=[

    ("text_count",CountVectorizer())

])
text_pipe3 = Pipeline(steps=[

    ("text_count",CountVectorizer())

])
text_pipe4 = Pipeline(steps=[

    ("text_count",CountVectorizer())

])
text_pipe5 = Pipeline(steps=[

    ("text_count",CountVectorizer())

])

#### Creating ColumnTransformer

In [ ]:
transformer = ColumnTransformer(transformers=[
    ("num_pipe",num_pipe,num_columns),
    ("cat_pipe",cat_pipe,cat_columns),
    ("text_pipe1",text_pipe1,"Title"),
    ("text_pipe2",text_pipe2,"Description"),
    ("text_pipe3",text_pipe3,"Status"),
    ("text_pipe4",text_pipe4,"Floor"),
    ("text_pipe5",text_pipe5,"overlooking")
    ],
    remainder='drop'

)

In [ ]:
transformer

### Model Hyperparameter tuning and evaluation using metrics

In [ ]:
#Creating a json dict for parameter tuning
model_tuning = {
    "svr":{
       "model" :SVR(),
       "params":{'kernel':['rbf','linear'],'degree':[3,4,5],'C':[1,10,50,100],'gamma':['scale', 'auto']}
    },
    "GBR":{
       "model" :GradientBoostingRegressor(),
       "params":{'n_estimators':[1,10,20,100],'learning_rate':[0.1,0.5,0.9]}
    },
    "RFR":{
       "model" :RandomForestRegressor(),
       "params":{'n_estimators':[10,50,100,120]}
    },
    "DTR":{
       "model" :DecisionTreeRegressor(),
       "params":{'splitter':["best"]}
    },
    "KNN":{
       "model" :KNeighborsRegressor(),
       "params":{'n_neighbors':[3,5,10,15]}
    }


}

In [ ]:
from sklearn.model_selection import GridSearchCV #for hyperparameter tuning

In [ ]:
model_score = []

In [ ]:
#A for loop to acess the model and it's parameters
for name,model_name in model_tuning.items():
    model1 = GridSearchCV(model_name['model'],model_name['params'],cv=5,return_train_score=False)
    model2 = make_pipeline(transformer,model1) #transformer for data preprocessing before fitting
    model2.fit(X_train,y_train)
    model_score.append({
        "":name,
        "best_performance":model1.best_score_,
        "best_params":model1.best_params_
    })


In [ ]:
pd.DataFrame(model_score) #Displaying a dataframe of the models performances and parameters

#### Make_pipeline to join or link the transformer and voting

In [ ]:
#Create the voting regressor for the models, using the best performing models above

In [ ]:
#SupportVectorRegressor,GradientBoostingRegressor,DTR,KNN and RandomForestRegressor performs best

voting = VotingRegressor(estimators=[
    ("svr",SVR(kernel='linear',C=100,gamma='auto')),
    ("gboost",GradientBoostingRegressor()),
    ("randomForest",RandomForestRegressor(n_estimators=120)),
    ("DTR",DecisionTreeRegressor()),
    ("KNN",KNeighborsRegressor())

])

In [ ]:
voting

In [ ]:
model = make_pipeline(transformer,voting) #making the pipeline

### LINKING THE TRANSFORMER AND VOTING PIPES USING make_pipeline

In [ ]:
# Training the model using fit
model.fit(X_train,y_train)

In [ ]:
model.score(X_val,y_val) #Model validation

## MODEL EVALUATION

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

In [ ]:
#Checking my model performance using my testing dataset

In [ ]:
y_predicted = model.predict(X_test)

In [ ]:
MAE = mean_absolute_error(y_test,y_predicted)
round(MAE,3)

### SAVING THE MODEL USING JOBLIB LIBRARY

In [ ]:
import joblib

In [ ]:
joblib.dump(model,"house_prediction.pkl")

In [ ]:
final_model = joblib.load("house_prediction.pkl")

##### Testing my final model

In [ ]:
final_model.predict(X_test[10:20]).tolist()

##### Expected answers of the unseen data, for testing the model

In [ ]:
y_test[10:20].tolist()

In [ ]:
final_model.score(X_test,y_test)